In [ ]:
#### demo_memorygame
#### memory card game, i.e., iterative delayed-match-to-sample
#### last updated 20181023 CRM

# experiment config

## logging variables
subID = 1

## card pictures for memory game
name_cardback = 'card'
# need seven icons, i.e., floor( {15 keys} / {pairs of 2} )
list_pictures = ['bulb', 'cheese', 'hammer', 'liberty', 'rocket', 'taco', 'turtle']

## run on actual device?
use_device = True


#### program code ####
from ElGateau import ElGateau, Icon
%matplotlib qt5
ElGateau = ElGateau(dev_mode=(not use_device),log='log_memorygame_sub{}'.format(subID))
ElGateau.display_clear('all')

# import experiment specific functions
import random, time

# prep variables
flips = 0
pairs_remain = len(list_pictures)

icon = {}
icon_list = list_pictures[:]
icon_list.append(name_cardback)
icon_list.append('cake')
for i in icon_list:
    icon[i]=Icon.prep(i,300)

# init key states
button_state_hidden = []
button_state_hidden.extend(list_pictures[:])
button_state_hidden.extend(list_pictures[:])
random.shuffle(button_state_hidden)
button_state_hidden.insert(0,'-') # placeholder to skip button 1

button_state_visible = ['-']
for button in range(2,16):
    button_state_visible.append(name_cardback)
    ElGateau.display_update(button,icon[button_state_visible[button-1]])

# begin experiment
while pairs_remain > 0:
    # update flip counter
    ElGateau.display_update(1, Icon.text(str(flips), size=32))
    
    # toggle state of first button
    valid = [i+1 for i, x in enumerate(button_state_visible) if x == name_cardback]
    button1, rt = ElGateau.button_listen_key(valid)
    ElGateau.display_update(button1,icon[button_state_hidden[button1-1]])

    # toggle state of second button
    valid.remove(button1)
    button2, rt = ElGateau.button_listen_key(valid)
    ElGateau.display_update(button2,icon[button_state_hidden[button2-1]])

    # update flip counter
    # value hasn't changed since last time, but forces screen update
    ElGateau.display_update(1, Icon.text(str(flips), size=32))

    # do buttons match?
    if button_state_hidden[button1-1] == button_state_hidden[button2-1]:
        # if they match, good job!
        pairs_remain -= 1
        button_state_visible[button1-1] = button_state_hidden[button1-1]
        button_state_visible[button2-1] = button_state_hidden[button2-1]
    else:
        # pause key presentation before re-hide
        time.sleep(1)
        # no match, toggle button state back
        ElGateau.display_update(button1,icon[button_state_visible[button1-1]])
        ElGateau.display_update(button2,icon[button_state_visible[button2-1]])

    flips += 1
    if hasattr(ElGateau,'Log'):
        # only if we're writing a log here
        ElGateau.Log.record_event('Flip {}'.format(flips))
    
# end
time.sleep(2)
ElGateau.display_clear('all')
ElGateau.display_update(8, Icon.text('Great\nJob', size=20))
ElGateau.display_update(13, Icon.prep('cake', 300))

# update flip counter
ElGateau.display_update(1, Icon.text(str(flips), size=32))

# close log
if hasattr(ElGateau,'Log'):
    ElGateau.Log.close()